In [1]:
import requests
from bs4 import BeautifulSoup
standings_url = "https://fbref.com/en/comps/9/2023-2024/2023-2024-Premier-League-Stats"
data = requests.get(standings_url, timeout=10)


In [2]:
soup = BeautifulSoup(data.text)
standings_table = soup.select('table.stats_table')[0] #select any table elements which have the class "stats table"

In [3]:
#to get all the href links

links = standings_table.find_all('a') #find all of the a tags, the a elements are the each of the team links
links = [l.get("href") for l in links]
links = [l for l in links if '/squads' in l] #just to keep the squad links to get stats for each individual squad

In [4]:
team_urls = [f"https://fbref.com{l}" for l in links]


team_url = team_urls[0]
data = requests.get(team_url)

In [5]:
print(len(data.text))

# Print a portion of the HTML to inspect it
print(data.text[:30000])  # Pri

893861
    
      
<!DOCTYPE html>
<html data-version="klecko-" data-root="/home/fb/deploy/www/base" lang="en" class="no-js" >
<head>
    <meta charset="utf-8">
    <meta http-equiv="x-ua-compatible" content="ie=edge">
    <meta name="viewport" content="width=device-width, initial-scale=1.0, maximum-scale=2.0" />
    <link rel="dns-prefetch" href="https://cdn.ssref.net/req/202407291" />
<script>
/* https://docs.osano.com/hc/en-us/articles/22469433444372-Google-Consent-Mode-v2  */
  window.dataLayer = window.dataLayer ||[];
      function gtag(){dataLayer.push(arguments);}
      gtag('consent','default',{
        'ad_storage':'denied',
        'analytics_storage':'denied',
        'ad_user_data':'denied',
        'ad_personalization':'denied',
        'personalization_storage':'denied',
        'functionality_storage':'granted',
        'security_storage':'granted',
        'wait_for_update': 500
      });
      gtag("set", "ads_data_redaction", true);
</script>
<script src="https://cmp

In [6]:
#grab the scores and fixtures table and turn into a df
import html5lib
import lxml
import pandas as pd

matches = pd.read_html(data.text, match="Scores & Fixtures ") #reads all tables on page, match looks for that string
matches[0].head()


/var/folders/30/dqf8s7ys1sq8x28l5wjh_3fc0000gn/T/ipykernel_44892/2891348857.py:6: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  matches = pd.read_html(data.text, match="Scores & Fixtures ") #reads all tables on page, match looks for that string


,Date,Time,Comp,Round,Day,Venue,Result,GF,GA,Opponent,xG,xGA,Poss,Attendance,Captain,Formation,Opp Formation,Referee,Match Report,Notes
0,2023-08-06,16:00,Community Shield,FA Community Shield,Sun,Neutral,D,1 (1),1 (4),Arsenal,NaN,NaN,55,81145.0,Kyle Walker,4-2-3-1,4-3-3,Stuart Attwell,Match Report,Arsenal won on penalty kicks following normal ...
1,2023-08-11,20:00,Premier League,Matchweek 1,Fri,Away,W,3,0,Burnley,1.9,0.3,65,21572.0,Kevin De Bruyne,4-2-3-1,5-4-1,Craig Pawson,Match Report,NaN
2,2023-08-16,22:00,Super Cup,UEFA Super Cup,Wed,Home,D,1 (5),1 (4),es Sevilla,NaN,NaN,74,NaN,Kyle Walker,4-2-3-1,4-2-3-1,François Letexier,Match Report,NaN
3,2023-08-19,20:00,Premier League,Matchweek 2,Sat,Home,W,1,0,Newcastle Utd,1.0,0.3,59,53419.0,Kyle Walker,4-2-3-1,4-3-3,Robert Jones,Match Report,NaN
4,2023-08-27,14:00,Premier League,Matchweek 3,Sun,Away,W,2,1,Sheffield Utd,3.5,0.7,79,31336.0,Kyle Walker,4-2-3-1,3-5-2,Jarred Gillett,Match Report,NaN


In [7]:
soup = BeautifulSoup(data.text)
links = soup.find_all('a')
links = [l.get("href") for l in links]
links = [l for l in links if l and 'all_comps/shooting/' in l]

In [8]:
data = requests.get(f"https://fbref.com{links[0]}") #will download html for the page
shooting = pd.read_html(data.text, match="Shooting")[0]

/var/folders/30/dqf8s7ys1sq8x28l5wjh_3fc0000gn/T/ipykernel_44892/3517699568.py:2: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  shooting = pd.read_html(data.text, match="Shooting")[0]


In [9]:
shooting.columns = shooting.columns.droplevel()

In [10]:
shooting.head()


,Date,Time,Comp,Round,Day,Venue,Result,GF,GA,Opponent,...,Dist,FK,PK,PKatt,xG,npxG,npxG/Sh,G-xG,np:G-xG,Match Report
0,2023-08-06,16:00,Community Shield,FA Community Shield,Sun,Neutral,D,1 (1),1 (4),Arsenal,...,NaN,NaN,0,0,NaN,NaN,NaN,NaN,NaN,Match Report
1,2023-08-11,20:00,Premier League,Matchweek 1,Fri,Away,W,3,0,Burnley,...,13.9,0.0,0,0,1.9,1.9,0.12,1.1,1.1,Match Report
2,2023-08-16,22:00,Super Cup,UEFA Super Cup,Wed,Home,D,1 (5),1 (4),es Sevilla,...,NaN,NaN,0,0,NaN,NaN,NaN,NaN,NaN,Match Report
3,2023-08-19,20:00,Premier League,Matchweek 2,Sat,Home,W,1,0,Newcastle Utd,...,17.9,0.0,0,0,1.0,1.0,0.07,0.0,0.0,Match Report
4,2023-08-27,14:00,Premier League,Matchweek 3,Sun,Away,W,2,1,Sheffield Utd,...,17.3,2.0,0,1,3.5,2.8,0.10,-1.5,-0.8,Match Report


In [11]:
#merge two dataframes tgt

team_data = matches[0].merge(shooting[["Date", "Sh", "SoT", "Dist", "FK", "PK", "PKatt"]], on="Date")
team_data.head()

,Date,Time,Comp,Round,Day,Venue,Result,GF,GA,Opponent,...,Opp Formation,Referee,Match Report,Notes,Sh,SoT,Dist,FK,PK,PKatt
0,2023-08-06,16:00,Community Shield,FA Community Shield,Sun,Neutral,D,1 (1),1 (4),Arsenal,...,4-3-3,Stuart Attwell,Match Report,Arsenal won on penalty kicks following normal ...,8,4,NaN,NaN,0,0
1,2023-08-11,20:00,Premier League,Matchweek 1,Fri,Away,W,3,0,Burnley,...,5-4-1,Craig Pawson,Match Report,NaN,17,8,13.9,0.0,0,0
2,2023-08-16,22:00,Super Cup,UEFA Super Cup,Wed,Home,D,1 (5),1 (4),es Sevilla,...,4-2-3-1,François Letexier,Match Report,NaN,23,7,NaN,NaN,0,0
3,2023-08-19,20:00,Premier League,Matchweek 2,Sat,Home,W,1,0,Newcastle Utd,...,4-3-3,Robert Jones,Match Report,NaN,14,4,17.9,0.0,0,0
4,2023-08-27,14:00,Premier League,Matchweek 3,Sun,Away,W,2,1,Sheffield Utd,...,3-5-2,Jarred Gillett,Match Report,NaN,29,9,17.3,2.0,0,1


In [12]:
matches[0].shape
shooting.shape

(58, 26)

In [13]:
years = list(range(2024,2022, -1))
years

[2024, 2023]

In [14]:
all_matches = []

In [15]:
standings_url = "https://fbref.com/en/comps/9/Premier-League-Stats"

In [17]:
import time
for year in years:
    data = requests.get(standings_url, timeout=10)
    soup = BeautifulSoup(data.text)
    standings_table = soup.select('table.stats_table')[0]


    links = [l.get("href") for l in standings_table.find_all('a')]
    links = [l for l in links if '/squads/' in l]
    team_urls = [f"https://fbref.com{l}" for l in links]

    previous_season = soup.select("a.prev")[0].get("href")
    standings_url = f"https://fbref.com{previous_season}"

    for team_url in team_urls:
        team_name = team_url.split("/")[-1].replace("-Stats", "").replace("-", " ")
        data = requests.get(team_url)
        matches = pd.read_html(data.text, match="Scores & Fixtures")[0]
        soup = BeautifulSoup(data.text)
        links = [l.get("href") for l in soup.find_all('a')]
        links = [l for l in links if l and 'all_comps/shooting/' in l]
        data = requests.get(f"https://fbref.com{links[0]}")
        shooting = pd.read_html(data.text, match="Shooting")[0]
        shooting.columns = shooting.columns.droplevel()
        try:
            team_data = matches.merge(shooting[["Date", "Sh", "SoT", "Dist", "FK", "PK", "PKatt"]], on="Date")
        except ValueError:
            continue
        team_data = team_data[team_data["Comp"] == "Premier League"]

        team_data["Season"] = year
        team_data["Team"] = team_name
        all_matches.append(team_data)
        time.sleep(10)

/var/folders/30/dqf8s7ys1sq8x28l5wjh_3fc0000gn/T/ipykernel_44892/778774674.py:18: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  matches = pd.read_html(data.text, match="Scores & Fixtures")[0]
/var/folders/30/dqf8s7ys1sq8x28l5wjh_3fc0000gn/T/ipykernel_44892/778774674.py:23: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  shooting = pd.read_html(data.text, match="Shooting")[0]
/var/folders/30/dqf8s7ys1sq8x28l5wjh_3fc0000gn/T/ipykernel_44892/778774674.py:18: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  matches = pd.read_html(data.text, match="Scores & Fixtures")[0]
/var/folders/30/dqf8s7ys1sq8x28l5wjh_3fc0000gn/T/ipykerne

In [20]:
match_df = pd.concat(all_matches)
match_df.columns = [c.lower() for c in match_df.columns]

match_df.to_csv("matches.csv")